In [1]:
%matplotlib inline

In [2]:
import os
from pathlib import Path

import numpy as np
import nibabel as nb
import nitransforms as nt

import plotly.express as px
import json
import re
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import cv2

from collections import defaultdict

In [3]:
patient_pattern = re.compile("([A-Z]*_[A-Z]*_[A-Z]*)_")

In [4]:
images_shapes = {}
mask_shapes = {}


images_arrays = {}
mask_arrays = {}

name_path = defaultdict(dict)

for image_path in Path("../data/files/").glob("*.nii"):
    
    
    image_name = image_path.name
    scan = nib.load(image_path)
    array = scan.get_fdata()
    shape = array.shape

    name = patient_pattern.findall(image_name)[0]
    
    if image_name.startswith("mask"):
        mask_shapes[name] = shape
        mask_arrays[name] = array
        
        name_path[name]["mask"] = image_path
    else:
        images_shapes[name] = shape
        images_arrays[name] = array

        name_path[name]["image"] = image_path

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [5]:
masks = set(mask_shapes.keys())
imgs = set(images_shapes.keys())

def format(set_data):
    return ',\n'.join(sorted(set_data))

has_only_mask = masks - imgs
has_only_imgs = imgs - masks

print(f"Есть только маски:\n{format(has_only_mask)}\n\n")
print(f"Есть только снимки:\n{format(has_only_imgs)}")

Есть только маски:
ANTONIAN_R_V,
BASHUROVA_YU_A,
BULLIEVA_L_F,
DYMOND_N_I


Есть только снимки:
ANTONYAN_R_V,
BAKHTIIAROVA_E_V,
BASHUROVA_IU_A,
CHUMAKOV_V_V


In [6]:
intersection = set(mask_shapes.keys()) & set(images_shapes.keys())

cols = ["id", "x1", "y1", "z1", "x2", "y2", "z2"]
data = []


for index, name in enumerate(intersection, 1):
    image_shape = np.array(images_shapes[name])
    mask_shape = np.array(mask_shapes[name])
    
    image_x, image_y, image_z = image_shape
    mask_x, mask_y, mask_z = mask_shape
    
    data.append([name, image_x, image_y, image_z, mask_x, mask_y, mask_z])
    
df = pd.DataFrame(data, columns=cols)

In [7]:
# px.scatter(df, x="id", y=["z1", "z2"])

In [8]:
intersection_names = sorted(intersection)

In [9]:
def save_images(path, name, mask=False):
    #Load the scan and extract data using nibabel 
    scan = nib.load(path)
    scanArray = scan.get_fdata()
    
    scanArrayShape = scanArray.shape
    print('The scan data array has the shape: ', scanArrayShape)

    outputPath = Path(f'{name}_mask/') if mask else Path(f'{name}/')
    outputPath.mkdir(parents=True, exist_ok=True)

    for i in range(scanArrayShape[2]):
        #Rotate slice clockwise 90 degrees
        outputArray = cv2.rotate(scanArray[:, :, i], cv2.ROTATE_90_CLOCKWISE)
        #Save the slice as .png image
        plt.imsave(str(outputPath) + f"/{i}.png", outputArray, cmap="grey")
    

In [10]:
# for name in intersection_names:
    
#     images_path = name_path[name]["image"]
#     mask_path = name_path[name]["mask"]
    
#     save_images(images_path, name, mask=False)
#     save_images(mask_path, name, mask=True)

In [11]:
def plot_image_and_mask(image, mask):
    fig, axs = plt.subplots(1,2)
    fig.suptitle('Image vs Mask. Nice?')
    axs[0].imshow(image, cmap='gray')
    axs[1].imshow(mask, cmap='gray')
    fig.tight_layout()
    return plt.show()

In [12]:
def crop_image(image, threshold=10):
    top_row = 0
    bottom_row = image.shape[1] - 1
    
    # Find the first non-black column from the left
    for i in range(image.shape[0]):
        if np.sum(image[i, :]) > threshold:
            top_row = i
            break
        
    # Find the first non-black col from the right
    for i in range(image.shape[0] - 1, -1, -1):
        if np.sum(image[i, :]) > threshold:
            bottom_row = i
            break
        
    cropped_image = image[top_row:bottom_row + 1, :]
    
    return cropped_image.astype(np.uint8)


def crop_image_and_mask(image, mask, threshold=10):
    top_row = 0
    bottom_row = image.shape[1] - 1
    
    # Find the first non-black column from the left
    for i in range(image.shape[0]):
        if np.sum(image[i, :]) > threshold:
            top_row = i
            break
        
    # Find the first non-black col from the right
    for i in range(image.shape[0] - 1, -1, -1):
        if np.sum(image[i, :]) > threshold:
            bottom_row = i
            break
        
    cropped_image = image[top_row:bottom_row + 1, :]
    cropped_mask = mask[top_row:bottom_row + 1, :]
    
    return cropped_image.astype(np.uint8), cropped_mask

In [13]:
def pad_image_and_mask_to_the_same_size(image, mask):
    
    def pad(array,  h_now, w_now, to_h, to_w):

        padded_array = np.zeros((to_h, to_w))
        
        padded_array[to_h-h_now:, to_w-w_now:] = array
                
        
        return padded_array
        
    
    image_h, image_w = image.shape
    mask_h, mask_w = mask.shape
    
    h = max(image_h, mask_h)
    w = max(image_w, mask_w)
    
    
    image_padded = pad(image, image_h, image_w, h, w)
    mask_padded = pad(mask, mask_h, mask_w, h, w)
    assert all((s_1 == s_2 for s_1, s_2 in zip(image_padded.shape, mask_padded.shape))), f"{image_padded.shape=} {mask_padded.shape=}"
    
    return image_padded, mask_padded

In [14]:
from IPython.display import clear_output, display

In [15]:
def merge_image_with_mask_and_draw(image, mask_binary):
    # global MASK, IMG
    
    # IMG = image
    # MASK = mask_binary
    rgb_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    rgb_image[mask_binary] = [255, 0, 0]
    return plt.imshow(rgb_image)

In [16]:
def shift_image(X, dx, dy):
    X = np.roll(X, dx, axis=1)
    X = np.roll(X, dy, axis=0)
    return X

In [17]:
def process_name(name: str, processed_data_path: Path):
    
    outdir = processed_data_path / "processed_data" / name
    outdir.mkdir(exist_ok=True, parents=True)
    
    print(name)
    
    images_path = processed_data_path / "raw_data" / name
    masks_path = processed_data_path / "raw_data" / f"{name}_mask"
    
    try:
        log = json.load(open(outdir / "log.json"))
        
    except FileNotFoundError:
        log = {"images_processed": 0}
        
    images_already_processed = log["images_processed"]
    
    number_of_images = len(list(images_path.glob("*.png")))
    
    print(f"Total number of slices is {number_of_images}")
    
    images_successful = images_already_processed
    for slice_index in range(images_already_processed, number_of_images):
        print(f"image {slice_index} / {number_of_images}")
        
        image = cv2.imread(str(images_path / f"{slice_index}.png"), 0)
        mask = cv2.imread(str(masks_path / f"{slice_index}.png"), 0)
        image_cropped = crop_image(image)
        
        # clear_output(True)
        # plot_image_and_mask(image_cropped, mask)
        # display(plt.gcf())
        # response = input("OK?")

        # if response == '-':
        #     continue
        
        image_cropped, mask = pad_image_and_mask_to_the_same_size(image_cropped, mask)
        
        mask = mask > 0
        image_cropped = image_cropped.astype(np.uint8)
        clear_output(True)
        merge_image_with_mask_and_draw(image_cropped, mask)
        display(plt.gcf())

        
        while True:
            offset_h, offset_w = map(int, input("Offsets H and W: ").split())
            
            shifted_mask = shift_image(mask, dx=offset_w, dy=offset_h)
            clear_output(True)
            merge_image_with_mask_and_draw(image_cropped, shifted_mask)
            display(plt.gcf())
            
            response = input("Shift OK?")

            if response == '-':
                break
            
            if response == '+':
                # save an image and a mask
                plt.imsave(str(outdir / f"{images_successful}_img.png"), image_cropped, cmap="grey")
                np.save(outdir / f"{images_successful}_mask", shifted_mask)
                
                images_successful += 1
                break
        
        log['images_processed'] += 1
        json.dump(log, open(outdir / "log.json", "w"))

In [18]:
# for name in intersection_names:
#     process_name(name, Path("../data"))

In [19]:
processed_dir = Path("../data/processed_data/")

name_to_samples = {}

for name_path in processed_dir.glob("./*"):
    name = name_path.name
    
    name_to_samples[name] = len(list(name_path.glob("*.png")))
    

In [20]:
overall_samples = sum(name_to_samples.values())
overall_samples

322

In [21]:
len(name_to_samples)

16

In [22]:
processed_data_root = Path("/home/fvelikon/projects/mri_cv/data/processed_data")

example_dir = processed_data_root / "ALIEV_S_B"
example_dir

PosixPath('/home/fvelikon/projects/mri_cv/data/processed_data/ALIEV_S_B')

In [25]:
# img = mask.astype(np.uint8) * 255
# img_canny = cv2.Canny(img, 50, 50)

# img_dilate = cv2.dilate(img_canny, None, iterations=1)
# img_erode = cv2.erode(img_dilate, None, iterations=1)

# mask_1 = np.full(img.shape, 255, "uint8")
# contours, hierarchies = cv2.findContours(img_erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
# for cnt in contours:
#     cv2.drawContours(mask_1, [cnt], -1, 0, -1)
    
# cv2.imshow("result", mask_1)
# cv2.waitKey(0)

In [56]:
heights = []
widths = []
cropped = []

for patient_dir in processed_dir.glob("./*"):
    
    
    for image_path in patient_dir.glob("*.png"):
        
        
        image = cv2.imread(str(image_path), 0)
        
        h_image, w_image = image.shape
        heights.append(h_image)
        widths.append(w_image)
        
        image = crop_image(image)
        
        h_image, w_image = image.shape
        heights.append(h_image)
        widths.append(w_image)
        
        cropped.extend([False, True])
    
df = pd.DataFrame(data={"width": widths, "height": heights, "cropped": cropped})
df.head()

,width,height,cropped
0,768,452,False
1,768,409,True
2,768,452,False
3,768,434,True
4,768,452,False


In [58]:
px.scatter(df, x="width", y="height", marginal_x="histogram", marginal_y="histogram", color="cropped")

In [59]:
px.density_contour(df, x="width", y="height", marginal_x="histogram", marginal_y="histogram", color="cropped")

In [23]:
import torch

In [78]:
processed_data_root_cropped = Path("/home/fvelikon/projects/mri_cv/data/processed_data_cropped")

for patient_dir in processed_dir.glob("./*"):
    
    patient_name = patient_dir.name
    
    
    new_patient_dir = processed_data_root_cropped / patient_name
    new_patient_dir.mkdir(exist_ok=True, parents=True)
    
    for new_index, image_path in enumerate(patient_dir.glob("*.png")):
        
        index = image_path.name.split("_")[0]
        
        mask_path = patient_dir / f"{index}_mask.npy"
        
        image = cv2.imread(str(image_path), 0)
        
        mask = np.load(mask_path, allow_pickle=True)
        
        image, mask = crop_image_and_mask(image, mask)
        
        assert all([dim_1 == dim_2 for dim_1, dim_2 in zip(image.shape, mask.shape)])

        
        resulting_array = torch.tensor(np.stack([image / 255, mask.astype(float)]))
        
        resulting_filename = new_patient_dir / f"{new_index}.pt"
        
        torch.save(resulting_array, resulting_filename)
        

In [27]:
processed_data_split_root = Path("/home/fvelikon/projects/mri_cv/data/processed_data_split")

processed_data_root_cropped = Path("/home/fvelikon/projects/mri_cv/data/processed_data_cropped")


train_dir = processed_data_split_root / "train"
test_dir = processed_data_split_root / "test"
eval_dir = processed_data_split_root / "val"

for _dir in [train_dir, eval_dir, test_dir]:
    _dir.mkdir(parents=True, exist_ok=True)

In [28]:
from shutil import copy

In [29]:
exclude_names = {"CHERVOV_A_G", "CHEBUNIN_N_P", "BARANOV_V_A", "BARKANOV_A_N"}

accepted_names = set(list([path.name for path in processed_data_root_cropped.glob("*")])) - exclude_names

test_names = set(np.random.choice(list(accepted_names), 2, replace=False))

accepted_names -= test_names

val_names = set(np.random.choice(list(accepted_names), 2, replace=False))

accepted_names -= val_names

train_names = accepted_names



counters = dict(zip(["train", "test", "eval"], [0, 0, 0]))

for patient_dir in processed_data_root_cropped.glob("./*"):
    
    name = patient_dir.name
    
    if name in exclude_names:
        continue
    
    
    if name in accepted_names:
        resulting_dir = train_dir
        
        mode = "train"
        
    elif name in test_names:
        resulting_dir = test_dir
        mode = "test"
        
    else:
        resulting_dir = eval_dir
        mode = "eval"
        
    

    for image_path in patient_dir.glob("*.pt"):
        
        resulting_name = resulting_dir / f"{counters[mode]}.pt"

        copy(image_path, resulting_name)
        
        counters[mode] += 1
        
        print(f"{name=} {mode=} {counters[mode]=}")
        

name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=1
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=2
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=3
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=4
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=5
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=6
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=7
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=8
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=9
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=10
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=11
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=12
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=13
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=14
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=15
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=16
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=17
name='CHERNOSKUTOVA_T_M' mode='train' counters[mode]=18
n